In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
#from scipy.spatial.transform import Rotation as R
import os
import matplotlib.animation as animation
import pandas as pd
#from PIL import Image
import matplotlib.image
from scipy.optimize import linear_sum_assignment
#import trackeval

#sequence='06'
#series = 'MOT_300K_4S_TR_1_LITE_06_0p3'

sequence='01'
series = 'MOT_300K_4S_TR_1_LITE'
true_fn='Z:/PHD/pcseg/TRACKS/KITTI_'+sequence+'.csv'
trax_fn='Z:/PHD/pcseg/results/'+series+'_TRACKING_H.csv'

ref = pd.read_csv(true_fn,header=None).to_numpy()
trk = pd.read_csv(trax_fn,header=None).to_numpy()

L_ref=np.unique(ref[:,0]).tolist()
L_trk=np.unique(trk[:,0]).tolist()
N_ref=len(L_ref)
N_trk=len(L_trk)
print(N_ref,N_trk,'\n')
print(L_ref)
print(L_trk)

NF_ref=int(np.max(ref[:,5]))
NF_trk=int(np.max(trk[:,14]))

NF=np.min([NF_ref,NF_trk])

print(NF_ref,NF_trk,NF,'\n')

trk_dst=[]

costm=np.zeros((N_ref,N_trk),dtype=np.float64)
costc=np.zeros((N_ref,N_trk),dtype=np.int32)

for F in range(NF):
    ref_F=ref[ref[:,5]==F,:]
    trk_F=trk[trk[:,14]==F,:]
    print(F,ref_F.shape[0],trk_F.shape[0])
    for i in range(ref_F.shape[0]):
        for j in range(trk_F.shape[0]):
            ii=L_ref.index(ref_F[i,0])
            jj=L_trk.index(trk_F[j,0])
            costm[ii,jj]+=np.linalg.norm([ref_F[i,2]-trk_F[j,8],ref_F[i,3]-trk_F[j,9],ref_F[i,4]-trk_F[j,10]])
            costc[ii,jj]+=1
            
for i in range(N_ref):
    for j in range(N_trk):
        if costc[i,j]>0:
            costm[i,j]/=float(costc[i,j])
            
for i in range(N_ref):
    for j in range(N_trk):
        if costm[i,j]<0.01:
            costm[i,j]=1000000000.0

row_ind, col_ind = linear_sum_assignment(costm)
print(row_ind)
print(col_ind)

for i in range(N_ref):
    ref_objid=L_ref[i]
    trk_trkid=L_trk[col_ind[i]]
    print(i,col_ind[i],ref_objid,trk_trkid,costc[i,col_ind[i]],costm[i,col_ind[i]])

# Plot 1 matching set of tracks (reference and prediction)
matchid=6 #2 #14

fig2,ax2 = plt.subplots(figsize=(10, 2.5))
figscale=1

for j in range(15,16):#range(N_ref):
    
    plt.clf()

    ref_objid=L_ref[j]
    trk_trkid=L_trk[col_ind[j]]

    ref_EX=ref[ref[:,0]==ref_objid,:]
    NP_ref_EX=ref_EX.shape[0]
    trk_EX=trk[trk[:,0]==trk_trkid,:]
    NP_trk_EX=trk_EX.shape[0]

    print(NP_ref_EX,NP_trk_EX)

    for i in range(NP_ref_EX-1):
        x1=ref_EX[i,3]
        y1=ref_EX[i,4]
        x2=ref_EX[i+1,3]
        y2=ref_EX[i+1,4]
        plt.plot([x1,x2], [y1,y2], '-', color=(0.9,0.1,0.1))
    
    for i in range(NP_trk_EX-1):
        x1=trk_EX[i,9]
        y1=trk_EX[i,10]
        x2=trk_EX[i+1,9]
        y2=trk_EX[i+1,10]
        plt.plot([x1,x2], [y1,y2], '-', color=(0.0,0.9,0.1))

    #plt.axis('equal')
    ax = plt.gca()
    ax.set_ylim([-3, -2])
    plt.grid()
    #plt.show()
    plt.savefig('Z:/PHD/pcseg/results/'+series+'_'+str(ref_objid)+'_TRK_COMPARE_V2.png')

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
#from scipy.spatial.transform import Rotation as R
import os
import matplotlib.animation as animation
import pandas as pd
#from PIL import Image
import matplotlib.image
from scipy.optimize import linear_sum_assignment
import trackeval

default_eval_config = trackeval.Evaluator.get_default_eval_config()
default_eval_config['DISPLAY_LESS_PROGRESS'] = False
default_dataset_config = trackeval.datasets.Kitti2DBox.get_default_dataset_config()
default_metrics_config = {'METRICS': ['HOTA', 'CLEAR', 'Identity']}
config = {**default_eval_config, **default_dataset_config, **default_metrics_config}  # Merge default configs

evaluator = trackeval.Evaluator(default_eval_config)
dataset_list = [trackeval.datasets.Kitti2DBox(default_dataset_config)]
metrics_list = []
for metric in [trackeval.metrics.HOTA, trackeval.metrics.CLEAR, trackeval.metrics.Identity]:
    if metric.get_name() in default_metrics_config['METRICS']:
        metrics_list.append(metric())
if len(metrics_list) == 0:
    raise Exception('No metrics selected for evaluation')

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
#from scipy.spatial.transform import Rotation as R
import os
import matplotlib.animation as animation
import pandas as pd
#from PIL import Image
import matplotlib.image
from scipy.optimize import linear_sum_assignment
import trackeval
from trackeval.datasets._base_dataset import _BaseDataset
from trackeval.metrics._base_metric import _BaseMetric
from trackeval.metrics.hota import HOTA
import motmetrics as mm

def Voxel_IoU(v1,v2):
    IoU = 0
    set1=set()
    for i in range(v1.shape[0]):
        voxel=(v1[i,0],v1[i,1],v1[i,2])
        set1.add(voxel)
    set2=set()
    for i in range(v2.shape[0]):
        voxel=(v2[i,0],v2[i,1],v2[i,2])
        set2.add(voxel)
        
    inter = set1 & set2
    union = set1 | set2
    if len(union)>0:
        IoU=len(inter)/len(union)
    
    return IoU

def VoxelPC_TrackerMC(cloud,rr,gg,bb,resolution):
    #cloud1 = pd.read_csv(M_fn_1,header=None).to_numpy()
    #cloud2 = pd.read_csv(M_fn_2,header=None).to_numpy()
    
    npt = cloud.shape[0]
    nc = 0
    for i in range(npt):
        if int(cloud[i,3])==rr and int(cloud[i,4])==gg and int(cloud[i,5])==bb:
            nc = nc + 1
            
    PC = np.zeros((nc,3))
    ni = 0
    for i in range(npt):
        if int(cloud[i,3])==rr and int(cloud[i,4])==gg and int(cloud[i,5])==bb:
            PC[ni,0]=cloud[i,0]
            PC[ni,1]=cloud[i,1]
            PC[ni,2]=cloud[i,2]
            ni = ni + 1
            
    return np.round(PC/resolution).astype(int)

def VoxelPC_GTMC(cloud,objid,resolution):
    #cloud1 = pd.read_csv(M_fn_1,header=None).to_numpy()
    #cloud2 = pd.read_csv(M_fn_2,header=None).to_numpy()
    
    npt = cloud.shape[0]
    nc = 0
    for i in range(npt):
        if int(cloud[i,0])==objid:
            nc = nc + 1
            
    PC = np.zeros((nc,3))
    ni = 0
    for i in range(npt):
        if int(cloud[i,0])==objid:
            PC[ni,0]=cloud[i,1]
            PC[ni,1]=cloud[i,2]
            PC[ni,2]=cloud[i,3]
            ni = ni + 1
            
    return np.round(PC/resolution).astype(int)

# 6
fn_gt = 'Z:\PHD\pcseg\TRACKS\KITTI_07_HOTA.csv'
fn_tr = 'Z:/PHD/pcseg/results/MOT_600K_4S_TR_1_LITE_07_0p3_SEGSHOTA_NC_F.csv'

# 1
#fn_gt = 'Z:\PHD\pcseg\TRACKS\KITTI_01_HOTA.csv'
#fn_tr = 'Z:/PHD/pcseg/results/MOT_300K_4S_TR_1_LITE_SEGSHOTA_NC.csv'

acc = mm.MOTAccumulator(auto_id=True)

# Load ground truth data
read_data, ignore_data = _BaseDataset._load_simple_text_file(fn_gt, time_col=5, id_col=0)

#print(read_data.keys())

tot=0
num_timesteps=len(read_data.keys())
print('GT Frames: ',num_timesteps)

data_keys = ['gt_ids', 'gt_dets']
raw_data = {key: [None] * num_timesteps for key in data_keys}

for k in read_data.keys():
    time_data = np.asarray(read_data[k], dtype=np.float)
    t=int(k)
    raw_data['gt_dets'][t] = np.atleast_2d(time_data[:, 7:13])
    raw_data['gt_ids'][t] = np.atleast_1d(time_data[:, 0]).astype(int)
    
    tot=tot+time_data.shape[0]
    #print(k,t,time_data.shape,raw_data['gt_dets'][t].shape,raw_data['gt_ids'][t].shape)

raw_data['gt_num_timesteps'] = num_timesteps
raw_data['seq'] = '0006'
print(tot)

# Load tracker data
read_data, ignore_data = _BaseDataset._load_simple_text_file(fn_tr, time_col=3, id_col=4)

#print(read_data.keys())

tot=0
num_timesteps=len(read_data.keys())
print('TR Frames: ',num_timesteps)
"""
for k in read_data.keys():
    time_data = np.asarray(read_data[k], dtype=np.float)
    print('Checking Key',k,int(k),time_data.shape)
    print(time_data[time_data[:,3]!=int(k),3])
"""
    

raw_data['tracker_ids'] = [None] * num_timesteps
raw_data['tracker_dets'] = [None] * num_timesteps
raw_data['tracker_confidences'] = [None] * num_timesteps
raw_data['similarity_scores'] = [None] * num_timesteps

for k in read_data.keys():
    time_data = np.asarray(read_data[k], dtype=np.float)
    t=int(k)
    raw_data['tracker_dets'][t] = np.atleast_2d(time_data[:, 5:11])
    raw_data['tracker_ids'][t] = np.atleast_1d(time_data[:, 4]).astype(int)
    raw_data['tracker_confidences'][t] = np.ones(time_data.shape[0])
    
    tot=tot+time_data.shape[0]
    #print(k,t,time_data.shape,raw_data['tracker_dets'][t].shape,raw_data['tracker_ids'][t].shape)
    
#print(read_data['195'])
#print(len(read_data['195']))

raw_data['tracker_num_timesteps'] = num_timesteps
print(tot)

total_tracked_dets = 0



for k in range(num_timesteps):
    t=str(k)
    IOU = _BaseDataset._calculate_3d_box_ious(raw_data['gt_dets'][k],raw_data['tracker_dets'][k])
    #print(IOU.shape)
    raw_data['similarity_scores'][k] = IOU
    #print(raw_data['gt_dets'][k].shape,raw_data['tracker_dets'][k].shape,raw_data['similarity_scores'][k].shape)
    
    acc.update(
    raw_data['gt_ids'][k],                     # Ground truth object ID in this frame
    raw_data['tracker_ids'][k],                # Detector hypothesis ID in this frame
    [IOU])
    
    """
    #cloud1 = pd.read_csv('Z:/PHD/pcseg/results/lrg/MOT_300K_4S_TR_1_LITE_06_0p3/M_'+str(k)+'.csv',header=None).values
    #cloud2 = pd.read_csv('Z:/PHD/pcseg/TRACKS/06_M/M_06_'+str(k)+'.csv',header=None).values
    cloud1 = pd.read_csv('Z:/PHD/pcseg/results/lrg/MOT_300K_4S_TR_1_LITE/M_'+str(k)+'.csv',header=None).values
    cloud2 = pd.read_csv('Z:/PHD/pcseg/TRACKS/01_M/M_01_'+str(k)+'.csv',header=None).values
    N1=cloud1.shape[0]
    N2=cloud2.shape[0]
    
    # Tracker
    s1=set()
    for i in range(N1):
        rgb=(int(cloud1[i,3]),int(cloud1[i,4]),int(cloud1[i,5]))
        s1.add(rgb)
    NC1=len(s1)
    #if k==195:
    #    print(s1)
    
    total_tracked_dets = total_tracked_dets + NC1

    # Ground Truth
    s2=set()
    for i in range(N2):
        objid=int(cloud2[i,0])
        s2.add(objid)
    NC2=len(s2)
    
    print(NC1,NC2,raw_data['tracker_dets'][k].shape[0],raw_data['gt_dets'][k].shape[0])
    if NC1!=raw_data['tracker_dets'][k].shape[0]:
        print(k,' Tracker Detection Count Mismatch',NC1-raw_data['tracker_dets'][k].shape[0])
    if NC2!=raw_data['gt_dets'][k].shape[0]:
        print(k,' Ground Truth Detection Count Mismatch',NC2-raw_data['gt_dets'][k].shape[0])
    
    print(k,N1,N2,NC1,NC2)

    IOU=np.zeros((NC2,NC1))

    voxels1=[]
    voxels2=[]

    for ss in s1:
        v=VoxelPC_TrackerMC(cloud1,ss[0],ss[1],ss[2],0.3)
        voxels1.append(v)
    
    for ss in s2:
        v=VoxelPC_GTMC(cloud2,ss,0.3)
        voxels2.append(v)


    for i in range(NC2):
        print(i)
        for j in range(NC1):
            IOU[i,j]=Voxel_IoU(voxels2[i],voxels1[j])
    
    raw_data['similarity_scores'][k] = IOU
    """

data_keys = ['gt_ids', 'tracker_ids', 'gt_dets', 'tracker_dets', 'tracker_confidences', 'similarity_scores']
data = {key: [None] * raw_data['tracker_num_timesteps'] for key in data_keys}
unique_gt_ids = []
unique_tracker_ids = []
num_gt_dets = 0
num_tracker_dets = 0
for t in range(raw_data['tracker_num_timesteps']):
    gt_ids = raw_data['gt_ids'][t]
    gt_dets = raw_data['gt_dets'][t]

    tracker_ids = raw_data['tracker_ids'][t]
    tracker_dets = raw_data['tracker_dets'][t]
    tracker_confidences = raw_data['tracker_confidences'][t]
    similarity_scores = raw_data['similarity_scores'][t]

    # Match tracker and gt dets (with hungarian algorithm)
    to_remove_matched = np.array([], np.int)
    unmatched_indices = np.arange(tracker_ids.shape[0])
    if gt_ids.shape[0] > 0 and tracker_ids.shape[0] > 0:
        matching_scores = similarity_scores.copy()
        matching_scores[matching_scores < 0.5 - np.finfo('float').eps] = 0
        match_rows, match_cols = linear_sum_assignment(-matching_scores)
        actually_matched_mask = matching_scores[match_rows, match_cols] > 0 + np.finfo('float').eps
        match_rows = match_rows[actually_matched_mask]
        match_cols = match_cols[actually_matched_mask]

        unmatched_indices = np.delete(unmatched_indices, match_cols, axis=0)

    unmatched_tracker_dets = tracker_dets[unmatched_indices, :]
    to_remove_tracker = to_remove_matched
    data['tracker_ids'][t] = np.delete(tracker_ids, to_remove_tracker, axis=0)
    data['tracker_dets'][t] = np.delete(tracker_dets, to_remove_tracker, axis=0)
    data['tracker_confidences'][t] = np.delete(tracker_confidences, to_remove_tracker, axis=0)
    similarity_scores = np.delete(similarity_scores, to_remove_tracker, axis=1)

    data['gt_ids'][t] = gt_ids
    data['gt_dets'][t] = gt_dets
    data['similarity_scores'][t] = similarity_scores

    unique_gt_ids += list(np.unique(data['gt_ids'][t]))
    unique_tracker_ids += list(np.unique(data['tracker_ids'][t]))
    num_tracker_dets += len(data['tracker_ids'][t])
    num_gt_dets += len(data['gt_ids'][t])
    
# Re-label IDs such that there are no empty IDs
if len(unique_gt_ids) > 0:
    unique_gt_ids = np.unique(unique_gt_ids)
    gt_id_map = np.nan * np.ones((np.max(unique_gt_ids) + 1))
    gt_id_map[unique_gt_ids] = np.arange(len(unique_gt_ids))
    for t in range(raw_data['tracker_num_timesteps']):
        if len(data['gt_ids'][t]) > 0:
            data['gt_ids'][t] = gt_id_map[data['gt_ids'][t]].astype(np.int)
if len(unique_tracker_ids) > 0:
    unique_tracker_ids = np.unique(unique_tracker_ids)
    tracker_id_map = np.nan * np.ones((np.max(unique_tracker_ids) + 1))
    tracker_id_map[unique_tracker_ids] = np.arange(len(unique_tracker_ids))
    for t in range(raw_data['tracker_num_timesteps']):
        if len(data['tracker_ids'][t]) > 0:
            data['tracker_ids'][t] = tracker_id_map[data['tracker_ids'][t]].astype(np.int)

# Record overview statistics.
data['num_tracker_dets'] = num_tracker_dets
data['num_gt_dets'] = num_gt_dets
data['num_tracker_ids'] = len(unique_tracker_ids)
data['num_gt_ids'] = len(unique_gt_ids)
data['num_timesteps'] = raw_data['tracker_num_timesteps']
data['seq'] = raw_data['seq']

"""
for t in range(raw_data['tracker_num_timesteps']):
    if len(data['gt_ids'][t]) > 0:
        print('GT: ',t,data['gt_ids'][t])
    if len(data['tracker_ids'][t]) > 0:
        print('TR: ',t,data['tracker_ids'][t])
"""

_BaseDataset._check_unique_ids(data)

hota=HOTA()
res=HOTA.eval_sequence(hota,data)
print('HOTA: ',np.mean(res['HOTA'])*100)
#print(data['num_tracker_dets'])
#print(total_tracked_dets)

#res['COMBINED_SEQ'] = {}
#HOTA.plot_single_tracker_results(res, 'MOT_300K_4S_TR_1_LITE_06_0p3', 'all', 'Z:\PHD\pcseg\results\')

# Use motmetrics for MOTA
mh = mm.metrics.create()
report = mh.compute(acc, metrics=['num_frames', 'num_objects','num_matches' ,'mota','motp', 'num_misses','num_false_positives','num_switches','mostly_tracked','partially_tracked','mostly_lost'], name='acc')
print(report)

GT Frames:  1101
24739
TR Frames:  1081
10076
HOTA:  5.785178284436374
     num_frames  num_objects  num_matches      mota      motp  num_misses  \
acc        1081        24010         8764  0.352187  0.000109       14242   

     num_false_positives  num_switches  mostly_tracked  partially_tracked  \
acc                  308          1004               5                100   

     mostly_lost  
acc           59  


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
#from scipy.spatial.transform import Rotation as R
import os
import matplotlib.animation as animation
import pandas as pd
#from PIL import Image
import matplotlib.image
from scipy.optimize import linear_sum_assignment
import trackeval
from trackeval.datasets._base_dataset import _BaseDataset
from trackeval.metrics._base_metric import _BaseMetric
from trackeval.metrics.hota import HOTA

gt=np.zeros((100,6))
tr=np.zeros((100,6))

for i in range(100):
    gt[i,0]=1.0
    gt[i,1]=1.0
    gt[i,2]=1.0
    gt[i,3]=2.0
    gt[i,4]=2.0
    gt[i,5]=2.0
    
    tr[i,0]=1.0
    tr[i,1]=1.0
    tr[i,2]=1.0
    tr[i,3]=2.0
    tr[i,4]=2.0
    tr[i,5]=2.05



scores = _BaseDataset._calculate_3d_box_ious(gt,tr)

print(scores)

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
#from scipy.spatial.transform import Rotation as R
import os
import matplotlib.animation as animation
import pandas as pd
#from PIL import Image
import matplotlib.image
from scipy.optimize import linear_sum_assignment
import trackeval
from trackeval.datasets._base_dataset import _BaseDataset
from trackeval.metrics._base_metric import _BaseMetric
from trackeval.metrics.hota import HOTA

def Voxel_IoU(v1,v2):
    IoU = 0
    set1=set()
    for i in range(v1.shape[0]):
        voxel=(v1[i,0],v1[i,1],v1[i,2])
        set1.add(voxel)
    set2=set()
    for i in range(v2.shape[0]):
        voxel=(v2[i,0],v2[i,1],v2[i,2])
        set2.add(voxel)
        
    inter = set1 & set2
    union = set1 | set2
    if len(union)>0:
        IoU=len(inter)/len(union)
    
    return IoU

def VoxelPC_TrackerMC(cloud,rr,gg,bb,resolution):
    #cloud1 = pd.read_csv(M_fn_1,header=None).to_numpy()
    #cloud2 = pd.read_csv(M_fn_2,header=None).to_numpy()
    
    npt = cloud.shape[0]
    nc = 0
    for i in range(npt):
        if int(cloud[i,3])==rr and int(cloud[i,4])==gg and int(cloud[i,5])==bb:
            nc = nc + 1
            
    PC = np.zeros((nc,3))
    ni = 0
    for i in range(npt):
        if int(cloud[i,3])==rr and int(cloud[i,4])==gg and int(cloud[i,5])==bb:
            PC[ni,0]=cloud[i,0]
            PC[ni,1]=cloud[i,1]
            PC[ni,2]=cloud[i,2]
            ni = ni + 1
            
    return np.round(PC/resolution).astype(int)

def VoxelPC_GTMC(cloud,objid,resolution):
    #cloud1 = pd.read_csv(M_fn_1,header=None).to_numpy()
    #cloud2 = pd.read_csv(M_fn_2,header=None).to_numpy()
    
    npt = cloud.shape[0]
    nc = 0
    for i in range(npt):
        if int(cloud[i,0])==objid:
            nc = nc + 1
            
    PC = np.zeros((nc,3))
    ni = 0
    for i in range(npt):
        if int(cloud[i,0])==objid:
            PC[ni,0]=cloud[i,1]
            PC[ni,1]=cloud[i,2]
            PC[ni,2]=cloud[i,3]
            ni = ni + 1
            
    return np.round(PC/resolution).astype(int)

cloud1 = pd.read_csv('Z:/PHD/pcseg/results/lrg/MOT_300K_4S_TR_1_LITE_06_0p3/M_0.csv',header=None).values
cloud2 = pd.read_csv('Z:/PHD/pcseg/TRACKS/06_M/M_06_0.csv',header=None).values
N1=cloud1.shape[0]
N2=cloud2.shape[0]
print(N1,N2)

# Tracker
s1=set()
for i in range(N1):
    rgb=(int(cloud1[i,3]),int(cloud1[i,4]),int(cloud1[i,5]))
    s1.add(rgb)
NC1=len(s1)

# Ground Truth
s2=set()
for i in range(N2):
    objid=int(cloud2[i,0])
    s2.add(objid)
NC2=len(s2)

IOU=np.zeros((NC1,NC2))

voxels1=[]
voxels2=[]

for ss in s1:
    v=VoxelPC_TrackerMC(cloud1,ss[0],ss[1],ss[2],0.3)
    voxels1.append(v)
    
for ss in s2:
    v=VoxelPC_GTMC(cloud2,ss,0.3)
    voxels2.append(v)


for i in range(NC1):
    print(i)
    for j in range(NC2):
        IOU[i,j]=Voxel_IoU(voxels1[i],voxels2[j])

print(np.max(IOU),np.min(IOU),np.mean(IOU))
            

In [2]:
import sys

dcp = 'C:\\Users\\paperspace\\AppData\\Local\\conda\\conda\\envs\\conda_env\\envs\\dcp_light_env\\Lib\site-packages'


if dcp not in sys.path:
    sys.path.append(dcp)

print(sys.path)
print(sys.executable)

['', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\python36.zip', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\DLLs', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib\\site-packages', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib\\site-packages\\win32', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib\\site-packages\\win32\\lib', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib\\site-packages\\Pythonwin', 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\paperspace\\.ipython', 'C:\\Users\\paperspace\\AppData\\Local\\conda\\conda\\envs\\conda_env\\envs\\dcp_light_env\\Lib\\site-packages']
C:\Pr